In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch import ConflictError

In [2]:
es_remote = Elasticsearch(["elasticsearch.ceda.ac.uk:443"], use_ssl=True, verify_certs=True)
remote_index = "c3s-roocs-fix-prop"

In [3]:
es_local = Elasticsearch(["rook-search5.cloud.dkrz.de:9200"])
 

In [4]:
from tqdm.notebook import tqdm
local_index = None
for doc in tqdm(scan(es_remote, index=remote_index), desc="reindex"):
    # print(doc["_source"]["dataset_id"])
    local_index = doc['_index']
    try:
        es_local.create(id=doc['_id'], index=local_index, document=doc['_source'])
    except ConflictError:
        pass

reindex: 0it [00:00, ?it/s]

/usr/local/Caskroom/mambaforge/base/envs/es/lib/python3.10/site-packages/elasticsearch/helpers/actions.py:576: ElasticsearchWarning: The client is unable to verify that the server is Elasticsearch due security privileges on the server side
  resp = client.search(**search_kwargs)
/usr/local/Caskroom/mambaforge/base/envs/es/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
es_local.indices.put_alias(index=local_index, name=remote_index)

{'acknowledged': True}